# The Classification Problem (Part 2)

## Instructions

-  Description of the dataset is available in the <code>**README.txt**</code>.

-  **GOAL:** Use information from past loan applicants contained in <code>**dataset.csv**</code> to predict whether a _new_ applicant should be granted a loan or not.

In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Import stats module from scipy, which contains a large number 
# of probability distributions as well as an exhaustive library of statistical functions.
import scipy.stats as stats

%matplotlib inline

## Summary of Part 1

In [2]:
# Path to the local dataset file
DATASET_PATH = "/Users/onuralpguvercin/Desktop/Padova Dersler/FIS python/lectures/Lecture_11_classification/loan-prediction/dataset.csv"

## Loading the Dataset

In [3]:
# Load the dataset with Pandas
data = pd.read_csv(DATASET_PATH, sep=",", index_col="Loan_ID")
print("Shape of the dataset: {}".format(data.shape))
data.head()
# NOTE: the first line of the file is considered as the header

Shape of the dataset: (614, 12)


,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
Loan_ID,,,,,,,,,,,,
LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


## Handling Missing Values (NA)

In [4]:
# is_numeric_dtype(pandas.Series) returns True iff the dtype associated with the pandas.Series is numeric
from pandas.api.types import is_numeric_dtype

data = data.apply(lambda x: x.fillna(x.median()) 
                      if is_numeric_dtype(x) 
                      else x.fillna(x.mode().iloc[0]))

## Handling Outliers

In [5]:
# Let's winsorize 'ApplicantIncome', 'CoapplicantIncome', and 'LoanAmount'
stats.mstats.winsorize(data.ApplicantIncome, limits=0.05, inplace=True)
stats.mstats.winsorize(data.CoapplicantIncome, limits=0.05, inplace=True)
stats.mstats.winsorize(data.LoanAmount, limits=0.05, inplace=True)

# Apply log-transformation to 'ApplicantIncome' and assign it to a new column
data['Log_ApplicantIncome'] = data.ApplicantIncome.apply(np.log)
# Apply log-transformation to 'LoanAmount' and assign it to a new column
data['Log_LoanAmount'] = data.LoanAmount.apply(np.log)

## Encoding Categorical Features: One-Hot Encoding

In [6]:
# In pandas we can achieve easily one-hot encoding using the 'get_dummies()' function
categorical_features = [col for col in data.columns if not is_numeric_dtype(data[col]) and col != 'Loan_Status']
data_with_dummies = pd.get_dummies(data, columns = categorical_features)
data_with_dummies.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status,Log_ApplicantIncome,Log_LoanAmount,Gender_Female,Gender_Male,...,Dependents_1,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban
Loan_ID,,,,,,,,,,,,,,,,,,,,,
LP001002,5849,0.0,128.0,360.0,1.0,Y,8.674026,4.852030,0,1,...,0,0,0,1,0,1,0,0,0,1
LP001003,4583,1508.0,128.0,360.0,1.0,N,8.430109,4.852030,0,1,...,1,0,0,1,0,1,0,1,0,0
LP001005,3000,0.0,66.0,360.0,1.0,Y,8.006368,4.189655,0,1,...,0,0,0,1,0,0,1,0,0,1
LP001006,2583,2358.0,120.0,360.0,1.0,Y,7.856707,4.787492,0,1,...,0,0,0,0,1,1,0,0,0,1
LP001008,6000,0.0,141.0,360.0,1.0,Y,8.699515,4.948760,0,1,...,0,0,0,1,0,1,0,0,0,1


In [7]:
# Just as a convention, I prefer to place the column to be predicted
# as the last one.
columns = data_with_dummies.columns.tolist()

# Popping out 'mpg' from the list and insert it back at the end.
columns.insert(len(columns), columns.pop(columns.index('Loan_Status')))

# Let's refactor the DataFrame using this new column index
data_with_dummies = data_with_dummies.loc[:, columns]
data_with_dummies.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Log_ApplicantIncome,Log_LoanAmount,Gender_Female,Gender_Male,Married_No,...,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban,Loan_Status
Loan_ID,,,,,,,,,,,,,,,,,,,,,
LP001002,5849,0.0,128.0,360.0,1.0,8.674026,4.852030,0,1,1,...,0,0,1,0,1,0,0,0,1,Y
LP001003,4583,1508.0,128.0,360.0,1.0,8.430109,4.852030,0,1,0,...,0,0,1,0,1,0,1,0,0,N
LP001005,3000,0.0,66.0,360.0,1.0,8.006368,4.189655,0,1,0,...,0,0,1,0,0,1,0,0,1,Y
LP001006,2583,2358.0,120.0,360.0,1.0,7.856707,4.787492,0,1,0,...,0,0,0,1,1,0,0,0,1,Y
LP001008,6000,0.0,141.0,360.0,1.0,8.699515,4.948760,0,1,1,...,0,0,1,0,1,0,0,0,1,Y


## Encoding Binary Class Label

In [8]:
data = data_with_dummies
data.Loan_Status = data.Loan_Status.map(lambda x: 1 if x=='Y' else -1)
data.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Log_ApplicantIncome,Log_LoanAmount,Gender_Female,Gender_Male,Married_No,...,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban,Loan_Status
Loan_ID,,,,,,,,,,,,,,,,,,,,,
LP001002,5849,0.0,128.0,360.0,1.0,8.674026,4.852030,0,1,1,...,0,0,1,0,1,0,0,0,1,1
LP001003,4583,1508.0,128.0,360.0,1.0,8.430109,4.852030,0,1,0,...,0,0,1,0,1,0,1,0,0,-1
LP001005,3000,0.0,66.0,360.0,1.0,8.006368,4.189655,0,1,0,...,0,0,1,0,0,1,0,0,1,1
LP001006,2583,2358.0,120.0,360.0,1.0,7.856707,4.787492,0,1,0,...,0,0,0,1,1,0,0,0,1,1
LP001008,6000,0.0,141.0,360.0,1.0,8.699515,4.948760,0,1,1,...,0,0,1,0,1,0,0,0,1,1


# 4. Building a Predictive Model

In [9]:
from sklearn.metrics import SCORERS
from sklearn.feature_extraction import DictVectorizer as DV
from sklearn import tree
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import explained_variance_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestRegressor
#from sklearn.externals import joblib

## 4.1 Splitting the Dataset: _Training_ vs. _Test_

In [10]:
"""
Extract the feature matrix from our original DataFrame.
"""
# Feature matrix X is composed of all the columns 
# except 'Loan_Status' (i.e., the target class label)
X = data.iloc[:, :-1]
X.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Log_ApplicantIncome,Log_LoanAmount,Gender_Female,Gender_Male,Married_No,...,Dependents_1,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban
Loan_ID,,,,,,,,,,,,,,,,,,,,,
LP001002,5849,0.0,128.0,360.0,1.0,8.674026,4.852030,0,1,1,...,0,0,0,1,0,1,0,0,0,1
LP001003,4583,1508.0,128.0,360.0,1.0,8.430109,4.852030,0,1,0,...,1,0,0,1,0,1,0,1,0,0
LP001005,3000,0.0,66.0,360.0,1.0,8.006368,4.189655,0,1,0,...,0,0,0,1,0,0,1,0,0,1
LP001006,2583,2358.0,120.0,360.0,1.0,7.856707,4.787492,0,1,0,...,0,0,0,0,1,1,0,0,0,1
LP001008,6000,0.0,141.0,360.0,1.0,8.699515,4.948760,0,1,1,...,0,0,0,1,0,1,0,0,0,1


In [11]:
"""
Similarly, we want to extract the target class column vector y.
"""
y = data.Loan_Status
y.head()

Loan_ID
LP001002    1
LP001003   -1
LP001005    1
LP001006    1
LP001008    1
Name: Loan_Status, dtype: int64

In [12]:
"""
Let's split our dataset with scikit-learn 'train_test_split' function, 
which splits the input dataset into training and test set, respectively.
We want the training set to account for 80% of the original dataset, whilst 
the test set to account for the remaining 20%.
Additionally, we would like to take advantage of stratified sampling,
so as to obtain the same target distribution in both the training and the test sets.
"""

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   test_size=0.2,
                                                   random_state=73,
                                                   stratify=y)

In [13]:
print("Training Set shape: {}".format(X_train.shape))
print("Test Set shape: {}".format(X_test.shape))

Training Set shape: (491, 22)
Test Set shape: (123, 22)


## Feature Scaling: Why/When

-  <span style="color: red">**REMEMBER:**</span> Not every learning models are sensitive to different feature scales! 

-  For example, in the case of Logistic Regression the vector of model parameters we come up with when we minimize the negative log-likelihood - using gradient descent (iterative) solution - is **not** affected by different feature scales, except for a constant.

-  You can convince yourself of this by computing the gradient of the negative log-likelihood using non-scaled and scaled features.

-  Other models, instead, are not invariant with respect to scalar transformations of the input (features), and leads to completely different results if features are not properly scaled (e.g., Support Vector Machines or SVM). 

## Feature Scaling: How

-  Feature scaling **cannot** be done looking at the whole dataset!

-  In other words, either you standardize (using **z-scores**) or normalize (using **min-max**) your features you **must** do it considering only the training set portion of your dataset.

-  The same scaling, then, should be applied to the test set.

In [14]:
from sklearn import preprocessing

# Standardizing features using z-score
std_scaler = preprocessing.StandardScaler().fit(X_train)
X_train_std = std_scaler.transform(X_train)
# Alternatively, using pure pandas:
# X_train_mean = X_train.mean()
# X_train_std = X_train.std()
# X_train_std = (X_train - X_train_mean)/X_train_std

# Normalizing features using min-max
minmax_scaler = preprocessing.MinMaxScaler().fit(X_train)
X_train_minmax = minmax_scaler.transform(X_train)
# Alternatively, using pure pandas:
# X_train_max = X_train.max()
# X_train_min = X_train.min()
# X_train_minmax = (X_train - X_train_min)/(X_train_max - X_train_min)

In [15]:
"""
At this stage, we can work with 3 different feature matrices:
- The original one: X_train
- The standardized one: X_train_std
- The min-max normalized one: X_train_minmax

In the following, however, we work only on the original feature matrix X_train
"""

'\nAt this stage, we can work with 3 different feature matrices:\n- The original one: X_train\n- The standardized one: X_train_std\n- The min-max normalized one: X_train_minmax\n\nIn the following, however, we work only on the original feature matrix X_train\n'

In [16]:
"""
General function used to assess the quality of predictions
in terms of two scores: accuracy and ROC AUC (Area Under the ROC Curve)
"""
def evaluate(true_values, predicted_values):
    # Classification Accuracy
    print("Accuracy = {:.3f}".
          format(accuracy_score(true_values, predicted_values)))
    # Explained variance score: 1 is perfect prediction
    print("Area Under the ROC Curve (ROC AUC) = {:.3f}".
          format(roc_auc_score(true_values, predicted_values)))

In [17]:
# Create logistic regression object
model = LogisticRegression(solver = "liblinear")

# 1. Try to fit this logistic regressor to our original training set
model.fit(X_train, y_train)

# 2. Assess the quality of predictions made on the same training set
print("***** Evaluate Performance on Training Set *****")
evaluate(y_train, model.predict(X_train))
print()

# 3. Assess the quality of predictions made on the test set
print("***** Evaluate Performance on Test Set *****") 
evaluate(y_test, model.predict(X_test))

***** Evaluate Performance on Training Set *****
Accuracy = 0.817
Area Under the ROC Curve (ROC AUC) = 0.717

***** Evaluate Performance on Test Set *****
Accuracy = 0.813
Area Under the ROC Curve (ROC AUC) = 0.712


In [18]:
print(classification_report(y_test, model.predict(X_test)))

              precision    recall  f1-score   support

          -1       0.89      0.45      0.60        38
           1       0.80      0.98      0.88        85

    accuracy                           0.81       123
   macro avg       0.85      0.71      0.74       123
weighted avg       0.83      0.81      0.79       123



In [19]:
# Simplest usage of cross-validation
model = LogisticRegression(solver = "liblinear")
cv = cross_validate(model, X, y, cv=10, scoring=('roc_auc', 'accuracy'), return_train_score=True)
pd.DataFrame(cv)

,fit_time,score_time,test_roc_auc,train_roc_auc,test_accuracy,train_accuracy
0,0.016044,0.007368,0.780906,0.794516,0.806452,0.809783
1,0.014619,0.008919,0.813953,0.793372,0.822581,0.809783
2,0.016387,0.007432,0.727381,0.805187,0.774194,0.813406
3,0.007800,0.005010,0.682143,0.809501,0.758065,0.817029
4,0.007547,0.005366,0.684211,0.809766,0.770492,0.813743
5,0.005676,0.003776,0.793233,0.795330,0.786885,0.811935
6,0.004609,0.003545,0.865915,0.787496,0.868852,0.802893
7,0.005618,0.003575,0.795739,0.798053,0.836066,0.804702
8,0.005775,0.003757,0.726817,0.803164,0.803279,0.806510
9,0.004871,0.004171,0.697995,0.806510,0.836066,0.810127


In [20]:
# Model evaluation using cross-validation
print("***** Evaluate Average Performance on Training Set *****")
print("Avg. Training Set Accuracy = {:.3f}".format(np.mean(cv['train_accuracy'])))
print("Avg. Training Set ROC AUC = {:.3f}".format(np.mean(cv['train_roc_auc'])))
print()
print("***** Evaluate Average Performance on Cross-Validation Set *****")
print("Avg. Test Set Accuracy = {:.3f}".format(np.mean(cv['test_accuracy'])))
print("Avg. Test Set ROC AUC = {:.3f}".format(np.mean(cv['test_roc_auc'])))

***** Evaluate Average Performance on Training Set *****
Avg. Training Set Accuracy = 0.810
Avg. Training Set ROC AUC = 0.800

***** Evaluate Average Performance on Cross-Validation Set *****
Avg. Test Set Accuracy = 0.806
Avg. Test Set ROC AUC = 0.757


In [21]:
# Define an object of type KFold and pass it to the cross_validate function
model = LogisticRegression(solver = "liblinear")

k_fold = KFold(n_splits=10, shuffle=True, random_state=42)

cv = cross_validate(model, X, y, cv=k_fold, scoring=('roc_auc', 'accuracy'), return_train_score=True)

pd.DataFrame(cv)

,fit_time,score_time,test_roc_auc,train_roc_auc,test_accuracy,train_accuracy
0,0.008324,0.005511,0.802555,0.795429,0.758065,0.815217
1,0.006948,0.005342,0.684504,0.812165,0.806452,0.809783
2,0.012558,0.013179,0.825000,0.792285,0.790323,0.811594
3,0.007876,0.006035,0.795594,0.797398,0.838710,0.806159
4,0.007687,0.007305,0.701449,0.808120,0.819672,0.810127
5,0.005668,0.003637,0.785714,0.795878,0.852459,0.808318
6,0.005537,0.003740,0.746867,0.801308,0.819672,0.808318
7,0.005177,0.004034,0.724806,0.806448,0.754098,0.815552
8,0.005278,0.003593,0.844048,0.792046,0.803279,0.808318
9,0.006404,0.003917,0.759420,0.797467,0.836066,0.806510


In [22]:
# Model evaluation using cross-validation
print("***** Evaluate Average Performance on Training Set *****")
print("Avg. Training Set Accuracy = {:.3f}".format(np.mean(cv['train_accuracy'])))
print("Avg. Training Set ROC AUC = {:.3f}".format(np.mean(cv['train_roc_auc'])))
print()
print("***** Evaluate Average Performance on Cross-Validation Set *****")
print("Avg. Test Set Accuracy = {:.3f}".format(np.mean(cv['test_accuracy'])))
print("Avg. Test Set ROC AUC = {:.3f}".format(np.mean(cv['test_roc_auc'])))

***** Evaluate Average Performance on Training Set *****
Avg. Training Set Accuracy = 0.810
Avg. Training Set ROC AUC = 0.800

***** Evaluate Average Performance on Cross-Validation Set *****
Avg. Test Set Accuracy = 0.808
Avg. Test Set ROC AUC = 0.767


In [23]:
# Define an object of type StratifiedKFold and pass it to the cross_validate function
model = LogisticRegression(solver = "liblinear")

k_fold = StratifiedKFold(n_splits=10, shuffle=True, random_state=37)

cv = cross_validate(model, X, y, cv=k_fold, scoring=('roc_auc', 'accuracy'), return_train_score=True)
print(cv)

{'fit_time': array([0.01115894, 0.00691295, 0.00872779, 0.00942421, 0.00650716,
       0.00868487, 0.0061121 , 0.00565505, 0.00541282, 0.00573897]), 'score_time': array([0.00631595, 0.00507379, 0.0040493 , 0.00638676, 0.00526881,
       0.0039382 , 0.00621796, 0.00426006, 0.00404811, 0.00464487]), 'test_roc_auc': array([0.7246022 , 0.74051408, 0.71309524, 0.78333333, 0.77318296,
       0.74686717, 0.77443609, 0.75814536, 0.69799499, 0.82205514]), 'train_roc_auc': array([0.80522214, 0.80436805, 0.80387087, 0.79891371, 0.79577122,
       0.79796167, 0.79668391, 0.80141466, 0.80503499, 0.79330697]), 'test_accuracy': array([0.85483871, 0.80645161, 0.79032258, 0.77419355, 0.75409836,
       0.83606557, 0.7704918 , 0.81967213, 0.78688525, 0.85245902]), 'train_accuracy': array([0.80434783, 0.8115942 , 0.80978261, 0.8134058 , 0.81735986,
       0.80650995, 0.81374322, 0.80650995, 0.80831826, 0.80470163])}


In [24]:
# Model evaluation using cross-validation
print("***** Evaluate Average Performance on Training Set *****")
print("Avg. Training Set Accuracy = {:.3f}".format(np.mean(cv['train_accuracy'])))
print("Avg. Training Set ROC AUC = {:.3f}".format(np.mean(cv['train_roc_auc'])))
print()
print("***** Evaluate Average Performance on Cross-Validation Set *****")
print("Avg. Test Set Accuracy = {:.3f}".format(np.mean(cv['test_accuracy'])))
print("Avg. Test Set ROC AUC = {:.3f}".format(np.mean(cv['test_roc_auc'])))

***** Evaluate Average Performance on Training Set *****
Avg. Training Set Accuracy = 0.810
Avg. Training Set ROC AUC = 0.800

***** Evaluate Average Performance on Cross-Validation Set *****
Avg. Test Set Accuracy = 0.805
Avg. Test Set ROC AUC = 0.753


## Model Selection and Evaluation

-  So far, we have just focused on a very specific _instance_ of a Logistic Regression model.

-  In other words, we haven't spent time trying to _tune_ any "meta-parameter" (known as **hyperparameter**) of our model.

-  In practice, we used default values of hyperparameters for our Logistic Regression model, according to <code>**scikit-learn**</code>

-  We didn't perform any actual model selection, as hyperparameters are fixed!

-  The figures we output for test accuracy/ROC AUC scores are our estimates of _generalization_ performance of our model (i.e., evaluation)

## Model Selection and Evaluation (cont'd)

-  Most of the time, though, we may need to do one of the following:
    1.  Fix a "family" of models (e.g., Logistic Regression) and perform hyperparameter selection;
    2.  Choose between a set of models (e.g., Logistic Regression, SVM, Decision Tree, etc.), each one with a fixed (i.e., default) set of hyperparameters;
    3.  A mixture of the above, where we have to select the best hyperparameters of the best model picked from a set of different models.

-  In any case, we also need to provide an estimate of the generalization performance of the chosen model.

## Select Best Model out of a Set of Family of Models with Fixed Hyperparameters

In [25]:
# Outer splitting: Training vs. Test set (e.g., 80÷20) used to separate training-selection-evaluation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=73, 
                                                    stratify=y)

# Inner splitting (i.e., within the outer training set): Training vs. Validation (e.g., 80÷20)
# Training set is used to train the model, validation set is used to select the best hyperparameters
X_train_train, X_validation, y_train_train, y_validation = train_test_split(X_train, y_train, test_size=0.2, 
                                                    random_state=42, 
                                                    stratify=y_train)

In [27]:
models = {'LogisticRegression': LogisticRegression(solver = "liblinear", max_iter=1000),
          'LinearSVC': LinearSVC(),
          'DecisionTreeClassifier': DecisionTreeClassifier(),
          'RandomForestClassifier': RandomForestClassifier(),
          'GradientBoostingClassifier': GradientBoostingClassifier()
          # Add other families of models here...
         }

In [28]:
k_fold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
cv_scores = {}
for model_name, model in models.items():
    cv_scores[model_name] = cross_val_score(model, X_train, y_train, cv=k_fold, scoring='accuracy')

/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number 

In [30]:
cv_df = pd.DataFrame(cv_scores).transpose()
cv_df

,0,1,2,3,4,5,6,7,8,9
LogisticRegression,0.82,0.755102,0.816327,0.816327,0.816327,0.816327,0.775510,0.795918,0.857143,0.836735
LinearSVC,0.52,0.612245,0.693878,0.591837,0.346939,0.693878,0.306122,0.673469,0.653061,0.408163
DecisionTreeClassifier,0.64,0.653061,0.693878,0.714286,0.734694,0.734694,0.673469,0.612245,0.714286,0.775510
RandomForestClassifier,0.76,0.734694,0.734694,0.857143,0.816327,0.816327,0.816327,0.795918,0.816327,0.816327
GradientBoostingClassifier,0.78,0.734694,0.734694,0.755102,0.775510,0.836735,0.755102,0.816327,0.755102,0.816327


In [31]:
cv_df['avg_cv'] = np.mean(cv_df, axis=1)
cv_df['std_cv'] = np.std(cv_df, axis=1)
cv_df = cv_df.sort_values(['avg_cv', 'std_cv'], ascending=[False,True])
cv_df

,0,1,2,3,4,5,6,7,8,9,avg_cv,std_cv
LogisticRegression,0.82,0.755102,0.816327,0.816327,0.816327,0.816327,0.775510,0.795918,0.857143,0.836735,0.810571,0.026278
RandomForestClassifier,0.76,0.734694,0.734694,0.857143,0.816327,0.816327,0.816327,0.795918,0.816327,0.816327,0.796408,0.036451
GradientBoostingClassifier,0.78,0.734694,0.734694,0.755102,0.775510,0.836735,0.755102,0.816327,0.755102,0.816327,0.775959,0.032585
DecisionTreeClassifier,0.64,0.653061,0.693878,0.714286,0.734694,0.734694,0.673469,0.612245,0.714286,0.775510,0.694612,0.045199
LinearSVC,0.52,0.612245,0.693878,0.591837,0.346939,0.693878,0.306122,0.673469,0.653061,0.408163,0.549959,0.133014


In [32]:
# Model Selection: Logistic Regression is the best overall method, therefore we pick that!
# Now we need to provide an estimate of its generalization performance. 
# To do so, we evaluate it against the test set portion we previously held out.
model = models[cv_df.index[0]]
# Re-fit the best selected model on the whole training set
model.fit(X_train, y_train)
# Evaluation
print("***** Evaluate Performance on Training Set *****")
evaluate(y_train, model.predict(X_train))
print("***** Evaluate Performance on Test Set *****")
evaluate(y_test, model.predict(X_test))

***** Evaluate Performance on Training Set *****
Accuracy = 0.817
Area Under the ROC Curve (ROC AUC) = 0.717
***** Evaluate Performance on Test Set *****
Accuracy = 0.813
Area Under the ROC Curve (ROC AUC) = 0.712


## Select the Best Hyperparameters AND the Best Model from a Family of Models

In [33]:
models_and_hyperparams = {'LogisticRegression': (LogisticRegression(),
                                                 {'C': [0.01, 0.05, 0.1, 0.5, 1, 2],
                                                 'penalty': ['l1', 'l2']}
                                                ),
                          'RandomForestClassifier': (RandomForestClassifier(),
                                       {'n_estimators': [10, 50, 100]}
                                       ),
                          'DecisionTreeClassifier': (DecisionTreeClassifier(),
                                                     {'criterion': ['gini', 'entropy'], 
                                                      'max_depth': [i for i in range(1, X.shape[1]+1)]}
                                                    )
                         }

In [34]:
# `outer_cv` creates 10 folds for estimating generalization error
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# when we train on a certain fold, we use a second cross-validation
# split in order to choose hyperparameters
inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=73)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=37, 
                                                    stratify=y)

# we will collect the average of the scores on the 10 outer folds in this dictionary
# with keys given by the names of the models in `models_and_hyperparams`
average_scores_across_outer_folds_for_each_model = dict()

In [35]:
average_scores_across_outer_folds_for_each_model

{}

In [36]:
my_list = []
# find the model with the best generalization error
for name, (model, params) in models_and_hyperparams.items():
    # this object is a classifier that also happens to choose
    # its hyperparameters automatically using `inner_cv`
    model_optimizing_hyperparams = GridSearchCV(estimator=model, 
                                                param_grid=params,
                                                cv=inner_cv, 
                                                scoring='accuracy',
                                                verbose=True)

    # estimate generalization error on the 10-fold splits of the data
    scores_across_outer_folds = cross_val_score(model_optimizing_hyperparams,
                                                X_train, y_train, cv=outer_cv, scoring='accuracy')

    # get the mean accuracy across each of outer_cv's 10 folds
    average_scores_across_outer_folds_for_each_model[name] = np.mean(scores_across_outer_folds)
    
    print(f'Model: {name}\nAccuracy in the 10 outer folds: {scores_across_outer_folds}.\nAverage Accuracy: {np.mean(scores_across_outer_folds)}')
    my_list.append(f'Model: {name}\nAccuracy in the 10 outer folds: {scores_across_outer_folds}.\nAverage Accuracy: {np.mean(scores_across_outer_folds)}')

Fitting 10 folds for each of 12 candidates, totalling 120 fits


/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

Fitting 10 folds for each of 12 candidates, totalling 120 fits


/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

...

Fitting 10 folds for each of 12 candidates, totalling 120 fits


/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

Model: LogisticRegression
Accuracy in the 10 outer folds: [0.76       0.89795918 0.81632653 0.81632653 0.83673469 0.85714286
 0.81632653 0.79591837 0.73469388 0.7755102 ].
Average Accuracy: 0.8106938775510203
Fitting 10 folds for each of 3 candidates, totalling 30 fits
Fitting 10 folds for each of 3 candidates, totalling 30 fits
Fitting 10 folds for each of 3 candidates, totalling 30 fits
Fitting 10 folds for each of 3 candidates, totalling 30 fits
Fitting 10 folds for each of 3 candidates, totalling 30 fits
Fitting 10 folds for each of 3 candidates, totalling 30 fits
Fitting 10 folds for each of 3 candidates, totalling 30 fits
Fitting 10 folds for each of 3 candidates, totalling 30 fits
Fitting 10 folds for each of 3 candidates, totalling 30 fits
Fitting 10 folds for each of 3 candidates, totalling 30 fits
Model: RandomForestClassifier
Accuracy in the 10 outer folds: [0.72       0.83673469 0.79591837 0.79591837 0.79591837 0.85714286
 0.83673469 0.7755102  0.69387755 0.75510204].
Avera

In [48]:
average_scores_across_outer_folds_for_each_model

{'LogisticRegression': 0.8106938775510203,
 'RandomForestClassifier': 0.7862857142857143,
 'DecisionTreeClassifier': 0.8087346938775509}

In [51]:
print('Average score across the outer folds: ',
      average_scores_across_outer_folds_for_each_model)

many_stars = '\n' + '*' * 100 + '\n'
print(many_stars + 'Now we choose the best model and refit on the whole dataset' + many_stars)

best_model_name, best_model_avg_score = max(average_scores_across_outer_folds_for_each_model.items(),
                                            key=(lambda name_averagescore: name_averagescore[1]))

# get the best model and its associated parameter grid
best_model, best_model_params = models_and_hyperparams[best_model_name]
best_model, best_model_params

Average score across the outer folds:  {'LogisticRegression': 0.8106938775510203, 'RandomForestClassifier': 0.7862857142857143, 'DecisionTreeClassifier': 0.8087346938775509}

****************************************************************************************************
Now we choose the best model and refit on the whole dataset
****************************************************************************************************



(LogisticRegression(),
 {'C': [0.01, 0.05, 0.1, 0.5, 1, 2], 'penalty': ['l1', 'l2']})

In [52]:
# now we refit this best model on the whole dataset so that we can start
# making predictions on other data, and now we have a reliable estimate of
# this model's generalization error and we are confident this is the best model
# among the ones we have tried
final_model = GridSearchCV(best_model, best_model_params, cv=inner_cv)
final_model.fit(X_train, y_train)

/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

Best model: 
	LogisticRegression()

Estimation of its generalization performance (accuracy):
	0.8106938775510203

Best parameter choice for this model: 
	{'C': 2, 'penalty': 'l2'}
(according to cross-validation `StratifiedKFold(n_splits=10, random_state=73, shuffle=True)` on the whole dataset).

              precision    recall  f1-score   support

          -1       0.80      0.47      0.59       192
           1       0.80      0.95      0.87       422

    accuracy                           0.80       614
   macro avg       0.80      0.71      0.73       614
weighted avg       0.80      0.80      0.78       614

Accuracy = [0.798]
Area Under the ROC = [0.788]


/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
60 fits failed out of a total of 120.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
60 fits failed with the following error:


In [53]:
print('Best model: \n\t{}'.format(best_model), end='\n\n')
print('Estimation of its generalization performance (accuracy):\n\t{}'.format(
    best_model_avg_score), end='\n\n')
print('Best parameter choice for this model: \n\t{params}'
      '\n(according to cross-validation `{cv}` on the whole dataset).'.format(
      params=final_model.best_params_, cv=inner_cv))


y_true, y_pred, y_pred_prob = y, final_model.predict(X), final_model.predict_proba(X)
print()
print(classification_report(y_true, y_pred))

roc = roc_auc_score(y_true, y_pred_prob[:,1])
acc = accuracy_score(y_true, y_pred)

print("Accuracy = [{:.3f}]".format(acc))
print("Area Under the ROC = [{:.3f}]".format(roc))

Best model: 
	LogisticRegression()

Estimation of its generalization performance (accuracy):
	0.8106938775510203

Best parameter choice for this model: 
	{'C': 2, 'penalty': 'l2'}
(according to cross-validation `StratifiedKFold(n_splits=10, random_state=73, shuffle=True)` on the whole dataset).

              precision    recall  f1-score   support

          -1       0.80      0.47      0.59       192
           1       0.80      0.95      0.87       422

    accuracy                           0.80       614
   macro avg       0.80      0.71      0.73       614
weighted avg       0.80      0.80      0.78       614

Accuracy = [0.798]
Area Under the ROC = [0.788]
